## Calculating Similarity
When calculating similarity between our transformer embedded vectors, we can use any of the three similarity metrics (euclidean, dot product, cosine).

But first, let's create some embeddings.

In [1]:
sentences = [
             "Three years later, the coffin was still full of Jello.",
             "The fish dreamed of escaping the fishbowl and into the toilet where he saw his friend go.",
             "The person box was packed with jelly many dozens of months later.",
             "Standing on one's head at job interviews forms a lasting impression.",
             "It took him a month to finish the meal.",
             "He found a leprechaun in his walnut shell."
]

In [3]:
from transformers import AutoTokenizer, AutoModel
import torch

tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')
model = AutoModel.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')

# initialize dictionary that will contain tokenized sentences
tokens = {'input_ids': [], 'attention_mask': []}

for sentence in sentences:
  # tokenize sentence and append to dictionary lists
  new_tokens = tokenizer.encode_plus(sentence, max_length=128, truncation=True,
                                     padding='max_length', return_tensors='pt')
  tokens['input_ids'].append(new_tokens['input_ids'][0])
  tokens['attention_mask'].append(new_tokens['attention_mask'][0])

# reformat list of tensors into single tensor
tokens['input_ids'] = torch.stack(tokens['input_ids'])
tokens['attention_mask'] = torch.stack(tokens['attention_mask'])

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

In [4]:
tokens['input_ids'].shape

torch.Size([6, 128])

We process these tokens through our model:


In [5]:
outputs = model(**tokens)
outputs.keys()

odict_keys(['last_hidden_state', 'pooler_output'])

The dense vector representations of our `text` are contained within the `outputs` 'last_hidden_state' tensor, which we access like so:

In [6]:
embeddings = outputs.last_hidden_state
embeddings

tensor([[[-6.9229e-02,  6.2300e-01,  3.5371e-02,  ...,  8.0334e-01,
           1.6314e+00,  3.2812e-01],
         [ 3.6730e-02,  6.8419e-01,  1.9460e-01,  ...,  8.4759e-02,
           1.4747e+00, -3.0080e-01],
         [-1.2141e-02,  6.5431e-01, -7.2716e-02,  ..., -3.2601e-02,
           1.7717e+00, -6.8121e-01],
         ...,
         [ 1.9532e-01,  1.1085e+00,  3.3905e-01,  ...,  1.2826e+00,
           1.0114e+00, -7.2754e-02],
         [ 9.0217e-02,  1.0288e+00,  3.2973e-01,  ...,  1.2940e+00,
           9.8650e-01, -1.1125e-01],
         [ 1.2404e-01,  9.7365e-01,  3.9329e-01,  ...,  1.1359e+00,
           8.7685e-01, -1.0435e-01]],

        [[-3.2124e-01,  8.2511e-01,  1.0554e+00,  ..., -1.8555e-01,
           1.5169e-01,  3.9366e-01],
         [-7.1457e-01,  1.0297e+00,  1.1217e+00,  ...,  3.3118e-02,
           2.3820e-01, -1.5632e-01],
         [-2.3522e-01,  1.1353e+00,  8.5941e-01,  ..., -4.3096e-01,
          -2.7242e-02, -2.9676e-01],
         ...,
         [-5.4000e-01,  3

In [7]:
embeddings.shape

torch.Size([6, 128, 768])

After we have produced our dense vetors `embeddings`, we need to perform a mean pooling operation on them to create a single vector encoding(the sentence embedding). To do this mean pooling operation we will need to multiply each value in our `embeddings` tensor by it's respective `attetion_mask` value - so that we ignore non-real tokens.

To perform this operation, we first resize our `attention_mask` tensor:

In [8]:
attention_mask = tokens['attention_mask']
attention_mask.shape

torch.Size([6, 128])

In [9]:
mask = attention_mask.unsqueeze(-1).expand(embeddings.size()).float()
mask.shape

torch.Size([6, 128, 768])

In [10]:
mask

tensor([[[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 

Each vector above represents a single token attention mask - each token now has a vector of size 768 representing it's attention_mask status. Then we multiply the two tensors to apply the attention mask:

In [11]:
masked_embeddings = embeddings * mask
masked_embeddings.shape

torch.Size([6, 128, 768])

In [12]:
masked_embeddings

tensor([[[-0.0692,  0.6230,  0.0354,  ...,  0.8033,  1.6314,  0.3281],
         [ 0.0367,  0.6842,  0.1946,  ...,  0.0848,  1.4747, -0.3008],
         [-0.0121,  0.6543, -0.0727,  ..., -0.0326,  1.7717, -0.6812],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000, -0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000, -0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000, -0.0000]],

        [[-0.3212,  0.8251,  1.0554,  ..., -0.1855,  0.1517,  0.3937],
         [-0.7146,  1.0297,  1.1217,  ...,  0.0331,  0.2382, -0.1563],
         [-0.2352,  1.1353,  0.8594,  ..., -0.4310, -0.0272, -0.2968],
         ...,
         [-0.0000,  0.0000,  0.0000,  ...,  0.0000, -0.0000,  0.0000],
         [-0.0000,  0.0000,  0.0000,  ...,  0.0000, -0.0000,  0.0000],
         [-0.0000,  0.0000,  0.0000,  ...,  0.0000, -0.0000,  0.0000]],

        [[-0.7576,  0.8399, -0.3792,  ...,  0.1271,  1.2514,  0.1365],
         [-0.6591,  0.7613, -0.4662,  ...,  0

Then we sum the remained of the embeddings along axis `1`:

In [13]:
summed = torch.sum(masked_embeddings, 1)
summed.shape

torch.Size([6, 768])

Then sum the number of values that must be given attetion in each position of the tensor:

In [14]:
summed_mask = torch.clamp(mask.sum(1), min=1e-9)
summed_mask.shape

torch.Size([6, 768])

In [15]:
summed_mask

tensor([[15., 15., 15.,  ..., 15., 15., 15.],
        [22., 22., 22.,  ..., 22., 22., 22.],
        [15., 15., 15.,  ..., 15., 15., 15.],
        [16., 16., 16.,  ..., 16., 16., 16.],
        [12., 12., 12.,  ..., 12., 12., 12.],
        [14., 14., 14.,  ..., 14., 14., 14.]])

Finally, we calculate the mean as the sum of the embedding activations `summed` divided by the number of values that should be given attention in each position `summed_mask`:

In [16]:
mean_pooled = summed / summed_mask

In [17]:
mean_pooled

tensor([[ 0.0745,  0.8637,  0.1795,  ...,  0.7734,  1.7247, -0.1803],
        [-0.3715,  0.9729,  1.0840,  ..., -0.2552, -0.2759,  0.0358],
        [-0.5030,  0.7950, -0.1240,  ...,  0.1441,  0.9704, -0.1791],
        [-0.0132,  0.9773,  1.4516,  ..., -0.8462, -1.4004, -0.4118],
        [-0.2019,  0.0597,  0.8603,  ..., -0.0100,  0.8431, -0.0841],
        [-0.2131,  1.0175, -0.8833,  ...,  0.7371,  0.1947, -0.3011]],
       grad_fn=<DivBackward0>)

And that is how we calculate our dense similarity vector.

In [18]:
from sklearn.metrics.pairwise import cosine_similarity

Let's calculate cosine similarity for sentence `0`:

In [19]:
# convert from PyTorch tensor to numpy array
mean_pooled = mean_pooled.detach().numpy()

# calculate
cosine_similarity([mean_pooled[0]], mean_pooled[1:])

array([[0.33088917, 0.7219258 , 0.17475505, 0.44709665, 0.5548363 ]],
      dtype=float32)

These similarities translate to:

| Index | Sentence | Similarity |
| --- | --- | --- |
| 1 | "The fish dreamed of escaping the fishbowl and into the toilet where he saw his friend go." | 0.3309 |
| 2 | "The person box was packed with jelly many dozens of months later." | 0.7219 |
| 3 | "Standing on one's head at job interviews forms a lasting impression." | 0.1748 |
| 4 | "It took him a month to finish the meal." | 0.4471 |
| 5 | "He found a leprechaun in his walnut shell." | 0.5548 |


So, as intended, the most similar sentence is that in index **2** - which contains the same meaning as our first sentence, without using the same words:

`"Three years later, the coffin was still full of Jello."`

## Embeddings with Sentence-Transformers
We've worked through creating our embeddings using the `transformers` library - and at times it can be quite involved. Now, it's important to understand the steps, but we can make life easier by using the `sentence-transformers` library. We will work through the same process - but using `sentence-transformers` instead.

In [20]:
sentences = [
             "Three years later, the coffin was still full of Jello.",
             "The fish dreamed of escaping the fishbowl and into the toilet where he saw his friend go.",
             "The person box was packed with jelly many dozens of months later.",
             "Standing on one's head at job interviews forms a lasting impression.",
             "It took him a month to finish the meal.",
             "He found a leprechaun in his walnut shell."
]

Initialize model:

In [24]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('bert-base-nli-mean-tokens')

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Encode the sentences:

In [25]:
sentence_embeddings = model.encode(sentences)

In [26]:
sentence_embeddings.shape

(6, 768)

In [27]:
sentence_embeddings

array([[ 0.07446172,  0.8636968 ,  0.17946427, ...,  0.77343976,
         1.7247487 , -0.18027495],
       [-0.3714633 ,  0.9729014 ,  1.0839936 , ..., -0.25521305,
        -0.2759372 ,  0.03575861],
       [-0.5029824 ,  0.7949859 , -0.12402495, ...,  0.14406337,
         0.970375  , -0.17911571],
       [-0.01324329,  0.9772858 ,  1.451594  , ..., -0.846165  ,
        -1.400432  , -0.41184366],
       [-0.2019263 ,  0.05970339,  0.8602745 , ..., -0.01000809,
         0.84306246, -0.08407697],
       [-0.21311918,  1.0174936 , -0.88327587, ...,  0.73710394,
         0.1946918 , -0.30111283]], dtype=float32)

And now we have our sentence embeddings - a much quicker approach. We then compare just as we did before using cosine similarity:

In [28]:
from sklearn.metrics.pairwise import cosine_similarity

Let's calculate cosine similarity for sentence `0`:

In [29]:
cosine_similarity([sentence_embeddings[0]], sentence_embeddings[1:])

array([[0.33088914, 0.7219258 , 0.17475505, 0.44709665, 0.5548363 ]],
      dtype=float32)

These similarities translate to almost the exact same values as we calculated before:

| Index | Sentence | Similarity (before) | New similarity |
| --- | --- | --- | --- |
| 1 | "The fish dreamed of escaping the fishbowl and into the toilet where he saw his friend go." | 0.3309 | 0.3309 |
| 2 | "The person box was packed with jelly many dozens of months later." | 0.7219 | 0.7219 |
| 3 | "Standing on one's head at job interviews forms a lasting impression." | 0.1748 | 0.174**7** |
| 4 | "It took him a month to finish the meal." | 0.4471 | 0.447**2** |
| 5 | "He found a leprechaun in his walnut shell." | 0.5548 | 0.554**7** |

So, using `sentence-transformers` can make life much easier. But either option produces the same outcome.